# Looping Through Pandas

- for loop with .iloc
- iterrows
- itertuple
- apply
- python zip
- pandas vectorization
- numpy vectorization

# Experiment results with %timeit

In [1]:
import timeit
import pandas as pd
import numpy as np

df = pd.DataFrame(np.random.randint(0, 10, size=(100000, 4)), columns=list('ABCD'))

In [3]:
df.sample(1)

,A,B,C,D
49533,6,9,5,9


## Standard python for loop with iloc
__A very basic way__ to achieve what we want to do is to use a standard for loop, and retrieve value using DataFrame's iloc method

In [4]:
def loop_with_for(df):
    temp = 0
    
    for index in range(len(df)):
        temp += df['A'].iloc[index] + df['B'].iloc[index]
        
    return temp

In [5]:
%timeit loop_with_for(df)

1.55 s ± 49.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%prun -l 4 loop_with_for(df)

10200009 function calls (10200008 primitive calls) in 3.128 seconds

Ordered by: internal time
List reduced from 38 to 4 due to restriction <4>

ncalls  tottime  percall  cumtime  percall filename:lineno(function)\
1600000    0.319    0.000    0.870    0.000 {built-in method builtins.isinstance}\
200000    0.276    0.000    0.494    0.000 frame.py:2893(__getitem__)\
200000    0.248    0.000    2.296    0.000 indexing.py:2205(_getitem_axis)\
400000    0.223    0.000    0.371    0.000 abc.py:180(__instancecheck__)\

__most of the time is spent on accessing values of each cell of the DataFrame, and checking data type with python’s isinstance function.__

## Using pandas iterrows function
__The pandas iterrows function returns a pandas Series for each row,__ with the down side of not preserving dtypes across rows.

In [7]:
def loop_with_iterrows(df):
    temp = 0
    
    for _, row in df.iterrows():
         temp += row.A + row.B
            
    return temp

In [8]:
%timeit loop_with_iterrows(df)

6.06 s ± 15.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%prun -l 4 loop_with_iterrows(df)

31400097 function calls (30900093 primitive calls) in 11.669 seconds

Ordered by: internal time
List reduced from 108 to 4 due to restriction <4>

ncalls  tottime  percall  cumtime  percall filename:lineno(function)\
4900013    1.095    0.000    2.495    0.000 {built-in method builtins.isinstance}\
4600006    1.009    0.000    1.218    0.000 {built-in method builtins.getattr}\
3000004    0.550    0.000    1.244    0.000 generic.py:7(_check)\
100000    0.413    0.000    8.078    0.000 series.py:152(__init__)\

__the iterrows approach is almost 5 times slow than using standard for loop! The reason, suggested by the above log, is that iterrows spends a lot of time creating pandas Series object, which is known to incur a fair amount of overhead. And yet, the Series it created does not preserve dtypes across rows.__

## Using pandas itertuples function

The pandas itertuples function is similar to iterrows, except it returns a namedtuple for each row, and preserves dtypes across rows.

In [10]:
def loop_with_itertuples(df):
    temp = 0
    
    for row_tuple in df.itertuples():
        temp += row_tuple.A + row_tuple.B
        
    return temp

In [11]:
%timeit loop_with_itertuples(df)

61.2 ms ± 475 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%prun -l 4 loop_with_itertuples(df)

301210 function calls (301195 primitive calls) in 0.107 seconds

Ordered by: internal time
List reduced from 113 to 4 due to restriction <4>

ncalls  tottime  percall  cumtime  percall filename:lineno(function)\
1    0.066    0.066    0.107    0.107 <ipython-input-10-8da54a617580>:1(loop_with_itertuples)\
100000    0.020    0.000    0.038    0.000 <string>:16(_make)\
100000    0.013    0.000    0.013    0.000 {built-in method __new__ of type object at 0x55c6b9ab7e00}\
100082/100068    0.005    0.000    0.005    0.000 {built-in method builtins.len}\
    
__itertuples saves the overhead of creating Series each row by creating namedtuple instead. This is efficient, yet we are still paying for overhead for creating namedtuple.__

## Using python zip
The way it works is it takes a number of iterables, and makes an iterator that aggragates elements from each of the iterables. Since a column of a Pandas DataFrame is an iterable, we can utilize zip to produce a tuple for each row just like itertuples, without all the pandas overhead!

In [14]:
def loop_with_zip(df):
    temp = 0
    
    for a, b in zip(df['A'], df['B']):
        temp += a + b
        
    return temp

In [15]:
%timeit loop_with_zip(df)

13.3 ms ± 297 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%prun -l 4 loop_with_zip(df)

140 function calls in 0.014 seconds

Ordered by: internal time
List reduced from 33 to 4 due to restriction <4>

ncalls  tottime  percall  cumtime  percall filename:lineno(function)\
1    0.013    0.013    0.014    0.014 <ipython-input-14-23370a87224c>:1(loop_with_zip)\
1    0.000    0.000    0.014    0.014 {built-in method builtins.exec}\
2    0.000    0.000    0.000    0.000 frame.py:2893(__getitem__)\
2    0.000    0.000    0.000    0.000 base.py:1117(__iter__)\
    
__The efficiency depends on the fact that we are not creating namedtuple for every row. zip simply returns an iterator of tuples.__

## Using pandas apply function

use the well-known pandas apply function, which is commonly used to do complex operations on DataFrame rows and columns.

In [17]:
def using_apply(df):
    return df.apply(lambda x: x['A'] + x['B'], axis=1).sum()

In [18]:
%timeit using_apply(df)

1.2 s ± 3.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%prun -l 4 using_apply(df)

8401971 function calls (8001949 primitive calls) in 2.585 seconds

Ordered by: internal time
List reduced from 218 to 4 due to restriction <4>

ncalls  tottime  percall  cumtime  percall filename:lineno(function)\
200000    0.296    0.000    1.663    0.000 base.py:4342(get_value)\
    1    0.183    0.183    2.583    2.583 {pandas._libs.reduction.reduce}\
200000    0.166    0.000    1.896    0.000 series.py:865(__getitem__)\
600297    0.135    0.000    0.313    0.000 {built-in method builtins.getattr}\

We are seeing about the same performance as using standard loops. __apply is still doing row operations and creating Series, which explains why the function calls are mostly getting values from Series.__ This is surprising as I always thought apply is one of the more efficient functions if one needs to do row operations.

## Using pandas builtin add function
using the pandas built-in add and sum function would have been the obvious way. __Unfortunately many computations we do does not have a simple built-in operation in Pandas. But this approach gives us a good indicator of how efficient these Pandas built-in functions are in practice.__

In [20]:
def using_pandas_builtin(df):
    return (df['A'] + df['B']).sum()

In [21]:
%timeit using_pandas_builtin(df)

660 µs ± 825 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [22]:
%prun -l 4 using_pandas_builtin(df)

655 function calls in 0.001 seconds

Ordered by: internal time
List reduced from 145 to 4 due to restriction <4>

ncalls  tottime  percall  cumtime  percall filename:lineno(function)\
3    0.000    0.000    0.000    0.000 {method 'reduce' of 'numpy.ufunc' objects}\
1    0.000    0.000    0.000    0.000 {built-in method _operator.add}\
1    0.000    0.000    0.000    0.000 {method 'copy' of 'numpy.ndarray' objects}\
1    0.000    0.000    0.000    0.000 missing.py:183(_isna_ndarraylike)\

Since pandas functions are highly optimized, it is expected to be very efficient. __This also shows that if your complex operation can be broken down to a series of pandas builtin functions, then it might make more sense to go that route than trying to jam all the operations__ in a lambda function then use apply.

## Using numpy builtin function
if you are really looking for efficiency and speed, always go to numpy. Here we convert each column into a numpy array, and does all the heavy lifting utilizing numpy’s builtin functionalities.

In [23]:
def using_numpy_builtin(df):
    return (df['A'].values + df['B'].values).sum()

In [24]:
%timeit using_numpy_builtin(df)

281 µs ± 70.2 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [25]:
%prun -l 4 using_numpy_builtin(df)

31 function calls in 0.001 seconds

Ordered by: internal time
List reduced from 19 to 4 due to restriction <4>

ncalls  tottime  percall  cumtime  percall filename:lineno(function)\
1    0.001    0.001    0.001    0.001 <ipython-input-23-0087d0c6165a>:1(using_numpy_builtin)\
1    0.000    0.000    0.000    0.000 {method 'reduce' of 'numpy.ufunc' objects}\
1    0.000    0.000    0.001    0.001 {built-in method builtins.exec}\
2    0.000    0.000    0.000    0.000 frame.py:2893(__getitem__)\
    
__the clear winner of this contest is the approach using numpy.__